In [1]:
import torch
import numpy as np
import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [0]:
import torchvision.datasets

In [3]:
MNIST_train = torchvision.datasets.MNIST('./', download = True, train = True)
MNIST_test = torchvision.datasets.MNIST('./', download = True, train = False)

0it [00:00, ?it/s]

9920512it [00:07, 1325036.51it/s]                             


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 57928.06it/s]                           
0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


1654784it [00:01, 967331.74it/s]                             
0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


8192it [00:00, 22051.04it/s]            

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [4]:
x_train = MNIST_train.train_data
y_train = MNIST_train.train_labels
x_test = MNIST_test.test_data
y_test = MNIST_test.test_labels

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [0]:
x_train = x_train.unsqueeze(1).float() #inserts singleton dim at position given as parameter
x_test = x_test.unsqueeze(1).float()

In [6]:
x_train.shape

torch.Size([60000, 1, 28, 28])

In [0]:
class LeNetMod(torch.nn.Module):
  def __init__(self):
    super(LeNetMod, self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, padding=1)
    self.act1 = torch.nn.ReLU()
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding=1)
    self.act2 = torch.nn.ReLU()

    self.conv3 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0)
    self.act3 = torch.nn.ReLU()
    self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = torch.nn.Linear(5 * 5 * 16, 120)
    self.act4 = torch.nn.ReLU()

    self.fc2 = torch.nn.Linear(120, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = self.act1(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = self.act2(x)

    x = self.conv3(x)
    x = self.act3(x)
    x = self.pool3(x)

    x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

    x = self.fc1(x)
    x = self.act4(x)
    x = self.fc2(x)

    return x

lenet_mod = LeNetMod()

In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet_mod.parameters(), lr=0.001)

In [22]:
batch_size = 100

test_accuracy_history = []
test_loss_history = []

for epoch in range(10000):
  order = np.random.permutation(len(x_train))
  for start_index in range(0, len(x_train), batch_size):
    optimizer.zero_grad()
    batch_indexes = order[start_index:start_index+batch_size]

    x_batch = x_train[batch_indexes]
    y_batch = y_train[batch_indexes]

    pred = lenet_mod.forward(x_batch)

    loss_value = loss(pred, y_batch)
    loss_value.backward()

    optimizer.step()

  test_pred = lenet_mod.forward(x_test)
  test_loss_history.append(loss(test_pred, y_test))

  accuracy = (test_pred.argmax(dim=1) == y_test).float().mean()
  test_accuracy_history.append(accuracy)

  print(accuracy)

tensor(0.9812)
tensor(0.9853)
tensor(0.9852)
tensor(0.9853)
tensor(0.9876)
tensor(0.9881)
tensor(0.9875)
tensor(0.9876)
tensor(0.9885)
tensor(0.9844)
tensor(0.9870)
tensor(0.9896)
tensor(0.9861)
tensor(0.9856)


KeyboardInterrupt: ignored